In [1]:
# importing required libraries
import pandas as pd
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn import naive_bayes
from sklearn.naive_bayes import MultinomialNB,BernoulliNB
from sklearn.metrics import roc_auc_score
import numpy as np
# importing pickle for model API
import pickle

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vinoths\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# opening the excel data
data1 = pd.read_excel('C:/FILES/TRAINING/Python training/Sentiment Analysis/APAC Apple & Android Verbatim Classification - Upto 01 July 2018.xlsx')

In [3]:
# Selecting only positive and negative categories, and changining positives to '1' and negetives to '0'
data1 = data1[(data1['Code1_Level1_Classification'] == 'Negative') | (data1['Code1_Level1_Classification'] == 'Positive')]
data1['Code1_Level1_Classification'] = data1['Code1_Level1_Classification'].replace('Negative','0') #changing negative values to zero
data1['Code1_Level1_Classification'] = data1['Code1_Level1_Classification'].replace('Positive','1') #changing positive values to one

In [4]:
# splitting training and testing data sets
train = data1[data1['Date']<='2018-05-31']
test = data1[data1['Date']>'2018-05-31']

In [5]:
# selecting required columns from data
cols = ['Q2','Code1_Level1_Classification']
train = train[cols]
test = test[cols]

In [6]:
# separating target variables
y_train = train['Code1_Level1_Classification']
y_test = test['Code1_Level1_Classification']

In [7]:
# intilising variable for removing stop words from the text and vectorize the words
stopset = set(stopwords.words('english'))
vectorizer = TfidfVectorizer(use_idf = True, lowercase = True, strip_accents = 'ascii', stop_words = stopset)

In [8]:
# vectorising x_train variables
x_train = vectorizer.fit_transform(train['Q2'])

In [9]:
# save the vectorizer model to disk
filename = 'C:/FILES/TRAINING/Python training/Sentiment Analysis/vocabulary.pkl'
#Save vectorizer.vocabulary_
pickle.dump(vectorizer.vocabulary_,open(filename ,"wb"))

In [10]:
# vectorising x_test variables
x_test = vectorizer.transform(test['Q2'])

In [11]:
# initialising and running multinomial naive bayes model on training data
clf = MultinomialNB()
clf.fit(x_train.toarray(),y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [12]:
# save the multinomialNB model to disk
filename = 'C:/FILES/TRAINING/Python training/Sentiment Analysis/sentimentMNB.sav'
pickle.dump(clf, open(filename, 'wb'))

In [13]:
# predicting the probabilites and class of the test data using the multinomialNB model
proba = clf.predict_proba(x_test.toarray())
pred = clf.predict(x_test.toarray())

In [14]:
# initialising and running bernoulli naive bayes model on training data
clf1 = BernoulliNB()
clf1.fit(x_train.toarray(),y_train)

BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)

In [15]:
# save the bernoulliNB model to disk
filename = 'C:/FILES/TRAINING/Python training/Sentiment Analysis/sentimentBNB.sav'
pickle.dump(clf1, open(filename, 'wb'))

In [16]:
# predicting the probabilites and class of the test data using the bernoulliNB model
proba0 = clf1.predict_proba(x_test.toarray())
pred0 = clf1.predict(x_test.toarray())

In [17]:
# finding the average probability of the test cases using both models
probability = (proba[:,1] + proba0[:,1]) /2

In [18]:
# applying cut-off (0.50) to the probabilities and arriving at the predicted class
prediction = np.where(probability < 0.5, 0, 1)

In [19]:
# displaying confusion matrix of the ensembled results
cfmatrix = pd.crosstab(y_test,prediction)
cfmatrix

col_0,0,1
Code1_Level1_Classification,,
0,751,19
1,47,321


In [20]:
# displaying accuracy of the ensemble model
diag = (cfmatrix.iloc[0,0]+cfmatrix.iloc[1,1]) # finding sum of diagonal
accuracy = diag/(diag+cfmatrix.iloc[0,1]+cfmatrix.iloc[1,0])

In [21]:
accuracy

0.9420035149384886

In [ ]:
# Scoring of new data

In [ ]:
#Getting input for scoring of the model
text = input("Enter text to find sentiment:")
text = [str(text)]

In [ ]:
#Changing the text into vectors
vector = vectorizer.transform(text)

In [ ]:
#Running model prediction
sentiment = clf.predict(vector)

In [ ]:
if sentiment[0] == '1':
    print ("Positive")
else: print ("Negative")

In [ ]:
# Saving the model for future scoring
# importing pickle for model API
import pickle

In [ ]:
# save the model to disk
filename = 'sentimentNPS.sav'
pickle.dump(model, open(filename, 'wb'))

In [ ]:
x_test